In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("data/Stockland-Visitation-Data-Consolidated-HC.xlsx")

In [3]:
df

,Asset,State,Post Code,Address,Unnamed: 4
0,Baldivis,WA,6171,"20 Settlers Ave, Baldivis WA 6171",NaN
1,Balgowlah,NSW,2093,"197-215 Condamine St, Balgowlah NSW 2093",NaN
2,Birtinya,QLD,4575,"8 The Avenue, Birtinya QLD 4575",NaN
3,Bull Creek,WA,6149,"South Street &, Benningfield Rd, Bull Creek WA...",NaN
4,Burleigh,QLD,4220,"149 W Burleigh Rd, Burleigh Waters QLD 4220",NaN
5,Gladstone,QLD,4680,"Dawson Hwy & Philip St, West Gladstone QLD 4680",NaN
6,Glendale,NSW,2285,"387 Lake Rd, Glendale NSW 2285",NaN
7,Green Hills,NSW,2323,"1 Molly Morgan Dr, East Maitland NSW 2323",NaN
8,Harrisdale,WA,6112,"Corner Nicholson Rd & Yellowwood Ave, Harrisda...",NaN
9,Hervey Bay,QLD,4655,"Stockland Hervey Bay, 6 Central Ave, Pialba QL...",NaN
